In [ ]:
import pandas as pd
import os
import requests
import json
import config
import datetime

In [ ]:
dataDir='data'

column_names=['ISBN',
             'Name',
             'Authors',
             'Description',
             'Language',
             'pagesNumber',
             'Publisher',
             'PublishYear',
             'Rating',
             'CountsOfReview']

mainData=pd.DataFrame(columns = column_names)

# Loop through each file in the data directory and load the files in a dataframe for ETL
for file in os.listdir(dataDir):
    filePath = '' + dataDir + '/' + os.fsdecode(file)
    df = pd.read_csv(filePath)
    # workaround for files without 'description' column
    if 'Description' not in df.columns:
        df['Description']="None"
    # initial stage of ETL - filter required columns
    df = df[['ISBN',
             'Name',
             'Authors',
             'Description',
             'Language',
             'pagesNumber',
             'Publisher',
             'PublishYear',
             'Rating',
             'CountsOfReview']]
    # remove non-english characters from Name and Author
    df['Name']=df['Name'].str.replace('[^a-zA-Z0-9!@#$%^&*()-+?/`~"\':; ]', '')
    #df['Authors']=df['Authors'].str.replace('[^a-zA-Z0-9!@#$%^&*()-+?/`~"\':; ]', '')
    # drop rows with missing values
    df.dropna(how='any',inplace=True)
    # append CSV data to main dataframe
    mainData = mainData.append(df,ignore_index=True)

In [ ]:
mainData

In [ ]:
# View distinct languages
mainData.Language.unique()

In [ ]:
# filter languages & drop duplicates
enLanguages=['en-US','eng','en-GB','en-CA']
mainData2=mainData[mainData.Language.isin(enLanguages)].copy()
# sorting will keep the records with most reviews when duplicates are dropped
mainData2.sort_values(by='CountsOfReview',ascending=0,inplace=True)
mainData2 = mainData[mainData.Language.isin(enLanguages)].drop_duplicates()
mainData2.drop_duplicates(subset='ISBN',inplace=True)
mainData2.drop_duplicates(subset='Name',inplace=True)
# Convert certain columns to INT
mainData2['CountsOfReview']=mainData2['CountsOfReview'].astype(int)
mainData2['pagesNumber']=mainData2['pagesNumber'].astype(int)
mainData2['PublishYear']=mainData2['PublishYear'].astype(int)
mainData2.info()

In [ ]:
mainData2.sort_values(by='Name',inplace=True)

In [ ]:
maxData = config.maximum_data
dataCut=mainData2.head(maxData).reset_index(drop=True)

In [ ]:
# initialise DF's
categoryDF = {"category_id":[],
             "category_name":[]}

isbn_categoryDF = {"isbn_no":[],
                   "category_id":[]}

authorDF = {"author_id":[],
            "author_name":[]}

isbn_authorDF = {"isbn_no":[],
                 "author_id":[]}

print_typeDF = {"print_type_id":[],
                "print_type":[]}

googlebooks_dataDF= {"isbn_no":[],
                     "print_type_id":[],
                     "retail_price":[]}
# get a list of ISBNs
isbn = dataCut['ISBN']

In [ ]:
# initialise ID's
category_id = 0
author_id = 0
print_type_id = 0

# initialise counters
prc_cntr=0
record=0
recs_fetched=1
set_no = 1

# create URL
url=f'https://www.googleapis.com/books/v1/volumes?key={config.g_key}&q=isbn:'

# record runtime
startTime = datetime.datetime.now().strftime('%d/%m/%y %H:%M:%S')

# loop through ISBNs and do a googlebooks API call
for i in isbn:

    # GET the API data
    response = requests.get(f"{url}{i}").json()
    prc_cntr += 1
    prcnt=round((prc_cntr/maxData)*100,0)

    # if response returns data then process the data
    if response['totalItems'] != 0:
        
        # initialise authors list
        authors=[]
        
        print(f"RECORD {prc_cntr}: {prcnt}% - Processing ISBN No. {i}")
        
        # get author data
        try:
            authors=response['items'][0]['volumeInfo']['authors']
        except (KeyError, IndexError):
            authors.append(dataCut.loc[dataCut['ISBN'] == i]["Authors"].iloc[0])
        
        # get print_type data
        print_type=response['items'][0]['volumeInfo']['printType']
        
        # get categories data
        try:
            categories=response['items'][0]['volumeInfo']['categories']
        except (KeyError, IndexError):
            categories=[]
        
        # get list price data
        try:
            listPrice=response['items'][0]['saleInfo']['listPrice']['amount']
        except (KeyError, IndexError):
            listPrice=0.00   
        
        # load categories data in objects
        if len(categories) > 0:
            for c in categories:
                cCaps = c.upper()
                if cCaps not in categoryDF['category_name']: 
                    category_id += 1
                    categoryDF['category_id'].append(category_id)
                    categoryDF['category_name'].append(cCaps)
                    finalCatId = category_id
                else: 
                    finalCatId = categoryDF['category_id'][categoryDF['category_name'].index(cCaps)]

                isbn_categoryDF['isbn_no'].append(i)
                isbn_categoryDF['category_id'].append(finalCatId)
        
        # load authors data in objects
        for a in authors:
            aCaps = a.upper()
            if aCaps not in authorDF['author_name']: 
                author_id += 1
                authorDF['author_id'].append(author_id)
                authorDF['author_name'].append(aCaps)
                finalAuthId = author_id
            else: 
                finalAuthId = authorDF['author_id'][authorDF['author_name'].index(aCaps)]

            isbn_authorDF['isbn_no'].append(i)
            isbn_authorDF['author_id'].append(finalAuthId)
        
        # load print type data
        if print_type not in print_typeDF['print_type']:
            ptCaps = print_type.upper()
            print_type_id += 1
            print_typeDF['print_type_id'].append(print_type_id)
            print_typeDF['print_type'].append(ptCaps)
            finalPrintId = print_type_id
        else:
            finalPrintId = print_typeDF['print_type_id'][print_typeDF['print_type'].index(ptCaps)]
        
        # load google books data
        googlebooks_dataDF['isbn_no'].append(i)
        googlebooks_dataDF['print_type_id'].append(finalPrintId)
        googlebooks_dataDF['retail_price'].append(listPrice)
        
    else:
        # skip if ISBN is not found
        print(f"RECORD {prc_cntr}: {prcnt}% - ISBN not found. Skipping...")

endTime = datetime.datetime.now().strftime('%d/%m/%y %H:%M:%S')

# record start and completion time
print(f"START TIME:     {startTime} \nCOMPLETION TIME: {endTime}")

In [ ]:
# Convert directory of lists to DataFrames
categoryDF=pd.DataFrame(categoryDF)
isbn_categoryDF=pd.DataFrame(isbn_categoryDF)
authorDF=pd.DataFrame(authorDF)
isbn_authorDF=pd.DataFrame(isbn_authorDF)
print_typeDF=pd.DataFrame(print_typeDF)
googlebooks_dataDF=pd.DataFrame(googlebooks_dataDF)